# Import Python packages

In [4]:
import os
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from scipy.io import loadmat


# Import Pyod Packages and Methods

In [6]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging


# Import Metrics Packages

In [8]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

# Define data file and read X and y

In [9]:
mat_file_list = [
    'arrhythmia.mat',
    'cardio.mat',
    'glass.mat',
    'ionosphere.mat',
    'letter.mat',
    'lympho.mat',
    'mnist.mat',
    'musk.mat',
    'optdigits.mat',
    'pendigits.mat',
    'pima.mat',
    'satellite.mat',
    'satimage-2.mat',
    'shuttle.mat',
    'vertebral.mat',
    'vowels.mat',
    'wbc.mat'
]

# Define nine outlier detection tools to be compared

In [12]:
df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc', 'ABOD', 'CBLOF', 'FB', 'HBOS', 'IForest', 'KNN', 'LOF', 'MCD', 'OCSVM', 'PCA']

roc_df = pd.DataFrame(columns = df_columns)
prn_df = pd.DataFrame(columns = df_columns)
time_df = pd.DataFrame(columns = df_columns)

# Input(Independenent) Feature Shape in Mat file format

In [15]:
data = loadmat('Anamoly_detec_data/arrhythmia.mat')
type(data['X']),data['X'].shape

(numpy.ndarray, (452, 274))

# Dependent/ Target/ Output Feature shape

In [17]:
type(data['y']),data['y'].shape

(numpy.ndarray, (452, 1))

# ROC (Region of Characterstics) Performance Evulotion table

In [19]:
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


# precision_n_scores - Performance evulotion table

In [21]:
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


# Time dataframe

In [23]:
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


# Exploring all Mat files

In [47]:
random_state = np.random.RandomState(42)
from time import time

for mat_file in mat_file_list:
    print("\n...Processing", mat_file, '....')
    mat = loadmat(os.path.join('Anamoly_detec_data', mat_file))
    
    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits = 4)
    
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
            
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = random_state)
    
    X_train_norm, X_test_norm = standardizer(X_train, X_test)
    
    classifiers = {
    'ABOD': ABOD(contamination=outliers_fraction),
    'CBLOF': CBLOF(contamination=outliers_fraction, check_estimator=False, random_state=random_state),
    'FeatureBagging': FeatureBagging(contamination=outliers_fraction, random_state=random_state),
    'HBOS': HBOS(contamination=outliers_fraction),
    'Isolation Forest': IForest(contamination=outliers_fraction, random_state=random_state),
    'KNN': KNN(contamination=outliers_fraction),
    'LOF': LOF(contamination=outliers_fraction),
    'MCD': MCD(contamination=outliers_fraction, random_state=random_state),
    'OCSVM': OCSVM(contamination=outliers_fraction),
    'PCA': PCA(contamination=outliers_fraction, random_state=random_state),
    }
    
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1-t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, execution time: {duration}s'.format(clf_name=clf_name, roc=roc, prn=prn, duration=duration))

        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis = 0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis = 0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis = 0)




...Processing arrhythmia.mat ....
ABOD ROC:0.7687, precision @ rank n:0.3571, execution time: 0.5766s
CBLOF ROC:0.7789, precision @ rank n:0.4643, execution time: 0.2618s
FeatureBagging ROC:0.7796, precision @ rank n:0.4643, execution time: 0.6266s
HBOS ROC:0.8511, precision @ rank n:0.5714, execution time: 0.0969s


C:\Users\rites\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


Isolation Forest ROC:0.8637, precision @ rank n:0.6071, execution time: 0.4794s
KNN ROC:0.782, precision @ rank n:0.5, execution time: 0.0919s
LOF ROC:0.7787, precision @ rank n:0.4643, execution time: 0.0674s


C:\Users\rites\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


MCD ROC:0.8228, precision @ rank n:0.4286, execution time: 1.0993s
OCSVM ROC:0.7986, precision @ rank n:0.5, execution time: 0.044s
PCA ROC:0.8, precision @ rank n:0.5, execution time: 0.068s

...Processing cardio.mat ....
ABOD ROC:0.5892, precision @ rank n:0.1918, execution time: 0.6846s
CBLOF ROC:0.8845, precision @ rank n:0.4932, execution time: 0.2149s
FeatureBagging ROC:0.6385, precision @ rank n:0.1781, execution time: 1.1763s
HBOS ROC:0.8373, precision @ rank n:0.4521, execution time: 0.027s


...

Isolation Forest ROC:0.3789, precision @ rank n:0.0, execution time: 0.3028s
KNN ROC:0.4573, precision @ rank n:0.0714, execution time: 0.016s
LOF ROC:0.4983, precision @ rank n:0.1429, execution time: 0.006s
MCD ROC:0.4103, precision @ rank n:0.0714, execution time: 0.066s
OCSVM ROC:0.4686, precision @ rank n:0.0714, execution time: 0.002s
PCA ROC:0.4085, precision @ rank n:0.0, execution time: 0.002s

...Processing vowels.mat ....
ABOD ROC:0.9616, precision @ rank n:0.6316, execution time: 0.5244s
CBLOF ROC:0.8963, precision @ rank n:0.3158, execution time: 0.1335s
FeatureBagging ROC:0.9365, precision @ rank n:0.3684, execution time: 0.4187s
HBOS ROC:0.6876, precision @ rank n:0.1579, execution time: 0.009s


C:\Users\rites\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


Isolation Forest ROC:0.8214, precision @ rank n:0.1579, execution time: 0.7721s
KNN ROC:0.9734, precision @ rank n:0.4737, execution time: 0.2322s
LOF ROC:0.9398, precision @ rank n:0.3684, execution time: 0.054s
MCD ROC:0.7243, precision @ rank n:0.1053, execution time: 1.4263s
OCSVM ROC:0.8163, precision @ rank n:0.2632, execution time: 0.057s
PCA ROC:0.6297, precision @ rank n:0.1579, execution time: 0.004s

...Processing wbc.mat ....
ABOD ROC:0.921, precision @ rank n:0.375, execution time: 0.1639s
CBLOF ROC:0.9149, precision @ rank n:0.375, execution time: 0.072s
FeatureBagging ROC:0.9271, precision @ rank n:0.375, execution time: 0.0929s
HBOS ROC:0.9479, precision @ rank n:0.5, execution time: 0.013s


C:\Users\rites\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


Isolation Forest ROC:0.9418, precision @ rank n:0.625, execution time: 0.2901s
KNN ROC:0.9444, precision @ rank n:0.5, execution time: 0.027s
LOF ROC:0.9227, precision @ rank n:0.375, execution time: 0.009s
MCD ROC:0.9288, precision @ rank n:0.5, execution time: 0.1119s
OCSVM ROC:0.9358, precision @ rank n:0.375, execution time: 0.007s
PCA ROC:0.9262, precision @ rank n:0.375, execution time: 0.004s
